In [1]:
import numpy as np
import pandas as pd

<div>
<img src="images/eurecat-logo.png" width="200"/>
</div>

# An informal Pyomo tutorial

### **_Biweekly_ · AAI Eurecat** 
June 4, 2024 \
_Giulia Zarpellon_

## 1. Optimization habits

**How do you do your optimization?**

Modeling language libraries, commercial software, solver-specific interfaces, ...

<div>
<img src="images/opt-zoo.png" width="750"/>
</div>

**How did I do my optimization?**

- Work directly with solvers (CPLEX, SCIP Optimization Suite) and modify the solution process quite heavily via _advanced callbacks_
- Often testing on benchmark optimization models -- _no concept of owning the "data process"_

**But outside academia things change:**
- More focus on proof of concepts and experimentation, prototypes and iterations: _we test more than one model_
- The data process is important: _sometimes we own it, sometimes not_
- We probably do not modify solvers settings too much... and first we need to figure out _which_ solver we want/can use

## 2. What is Pyomo?

<div>
<img src="images/pyomo-logo.webp" width="250"/>
</div>

- Optimization modeling language
- Python-based
- Open-source
- Customizable
- Solver-agnostic

&#8594; [Pyomo documentation](https://pyomo.readthedocs.io/en/stable/index.html)

**What Pyomo is _NOT_...**

- An optimization solver: _Pyomo is an interface to solvers_

## 3. Why / when using Pyomo?

- You want to quickly start building an optimization model in iterations
- You have to test different model formulations
- Solver choice is open / undecided
- No need for advanced callbacks
- Integration with Python is important (e.g., for visualizations)

_All these applied for the **DiOptiBa** project, so we went for it!_

**Available solvers / software interfaces**: 
- _(commercial)_ CPLEX, Gurobi, Express, GAMS, AMPL, Mosek
- _(CoinOR)_ IPopt, CBC
- _(others)_ [HiGHS](https://highs.dev/), MAiNGO, BARON, GLPK, CONOPT, Scipy, ...

Interfaces can be specialized or generic (read an AMPL `.nl` file and write a `.sol` file). 

And many more! Some are available through the [NEOS server](https://neos-server.org/neos/) (asynchronous, does not require a solver locally installed)

Installation usually runs smoothly:

````
pip install 'pyomo[optional]'     # install pyomo with its conditional dependencies to get interface with HiGHS
pip install highspy               # install HiGHS (or another solver of choice)
````

## 4. Pyomo 101

### Model components and process

Object-oriented design of models, using as components

- ``Set`` · data that needs to be organized in a set, e.g., indices
- ``Param`` · parameter data that will be specified as model input
- ``Var`` · model decision variables
- ``Constraint`` · expressions that enforce restrictions in the solution space
- ``Objective`` · expressions that should be maximized or minimized

Typical steps when modeling:

1. Define the model by declaring its components
2. Instantiate the model with data
3. Apply solver (i.e., run the optimization algorithm)
4. Interrogate solver results

### Concrete vs. abstract models

Often the biggest distinction made when discussing Pyomo implementations

&#8594; `ConcreteModel` : _"data first"_, data is specified at the time of model definition

&#8594; `AbstractModel` : _"model first"_, data will be supplied later, when the model is solved

Which one should you choose?

**The truth is...**

- there is no "better" option, even though in some community forums `ConcreteModel` seems the preferred way to go
- it depends on the use-case and data specification
- it depends on your preferences and approach

**... there's a spectrum of options available for how to specify models**

## 5. Examples

### Production mix model

You own a home-based bakery gig where you make only two types of items:

<div>
<img src="images/loaf-tray.png" width="600"/>
</div>

**Problem specification and data**

- You need to decide the number of units of bread and pizza to produce each week
- Production is constrained by
    - How many hours of work you have to put in
    - How much total flour you have available
    - The sale statistics, which tell you that production of pizza trays should be less than or equal to twice the production of bread
- Your objective is to maximize gross profit margin per week

Note: _This data is * not * realistic_ 

|  | Bread loaf | Pizza tray |
| -------- | ------- | ------- |
| Time | 8.5h | 4h |
| Flour | 1.8kg | 3kg |
| Profit | 8€ | 20€ |

In [2]:
import pyomo.environ as pyo
from pyomo.contrib import appsi

### Model 1 · Concrete model with explicit data

a.k.a. when you just need a quick way to test a small model

In [3]:
model1 = pyo.ConcreteModel(name='bread&pizza_model1')

# define variables
model1.loafs = pyo.Var(domain=pyo.NonNegativeIntegers)
model1.trays = pyo.Var(domain=pyo.NonNegativeIntegers)

# define constraints
model1.time_constraint = pyo.Constraint(
    expr=8.5 * model1.loafs + 4 * model1.trays <= 120)
model1.flour_constraint = pyo.Constraint(
    expr=1.8 * model1.loafs + 3.0 * model1.trays <= 50)
model1.sales_constraint = pyo.Constraint(
    expr=-2.00 * model1.loafs + 1.00 * model1.trays <= 0)

# define objective
model1.profit_obj = pyo.Objective(
    expr=8.00 * model1.loafs + 20.00 * model1.trays, sense=pyo.maximize)

- **Data is hard-coded** into the model definition
- Very direct approach, but very rigid (one by one definition of variables)
- May be OK for very small models, but not for bigger ones

In [4]:
# solve
opt = appsi.solvers.Highs()  # call optimizer of choice
results = opt.solve(model1)  # optimize and store results

# output some results
print(f"{model1.name}")
print(f"Status: {results.termination_condition}\n")
print(f"Total profit: {np.round(model1.profit_obj(), 2)}€")
print(f"# loafs: {model1.loafs()}")
print(f"# trays: {model1.trays()}")

bread&pizza_model1
Status: TerminationCondition.optimal

Total profit: 296.0€
# loafs: 7.0
# trays: 12.0


### Model 2 · Concrete model, separate data

- Use **Python data-structures** to specify data and pass it to the Pyomo model
- In this example we use a dictionary, but data can come in other forms (DataFrames, json, ...)

In [5]:
# define data separately
data = {
    "name": "bread&pizza_model2",
    "max_hours": 120,
    "max_kgs": 50,
    "sales_limit": 0,
    "coefficients": {
        'loafs': {'hours': 8.5, 'flour': 1.8, 'sales': -2.00, 'profit': 8},  
        'trays': {'hours': 4, 'flour': 3, 'sales':  1.00, 'profit': 20},
    },
    "products_init": 0,
    "products_lb": 0, 
    "products_ub": 100
}

In [6]:
model2 = pyo.ConcreteModel(name=data['name'])

# define scalar parameters
model2.max_hours = pyo.Param(
    within=pyo.NonNegativeReals, initialize=data['max_hours'])
model2.max_kgs = pyo.Param(
    within=pyo.NonNegativeReals, initialize=data['max_kgs'])
model2.sales_limit = pyo.Param(
    within=pyo.NonNegativeReals, initialize=data['sales_limit'])
model2.products_init = pyo.Param(
    within=pyo.NonNegativeReals, initialize=data['products_init'])
model2.products_lb = pyo.Param(
    within=pyo.NonNegativeReals, initialize=data['products_lb'])
model2.products_ub = pyo.Param(
    within=pyo.NonNegativeReals, initialize=data['products_ub'])

In [7]:
# define a Pyomo set to index products
coefficients = data['coefficients']
model2.products = pyo.Set(initialize=list(coefficients.keys()))

# and additional parameters that are indexed by the set
# mutable: allowed to change after initialization
model2.hours = pyo.Param(
    model2.products, within=pyo.NonNegativeReals, mutable=True)  
model2.flour = pyo.Param(
    model2.products, within=pyo.NonNegativeReals, mutable=True)
model2.sales = pyo.Param(model2.products, within=pyo.Reals, mutable=True)
model2.profit = pyo.Param(model2.products, within=pyo.Reals, mutable=True)

for p in model2.products:    
    model2.hours[p] = coefficients[p]['hours']
    model2.flour[p] = coefficients[p]['flour']
    model2.sales[p] = coefficients[p]['sales']
    model2.profit[p] = coefficients[p]['profit']

In [8]:
# define variables, also indexed by the products Set
model2.production = pyo.Var(
    model2.products, domain=pyo.NonNegativeIntegers, 
    initialize=model2.products_init, 
    bounds=(model2.products_lb, model2.products_ub)
)

# define constraints
model2.time_constraint = pyo.Constraint(
    expr=sum(model2.hours[p] * model2.production[p] for p in model2.products) 
    <= model2.max_hours)
model2.flour_constraint = pyo.Constraint(
    expr=sum(model2.flour[p] * model2.production[p] for p in model2.products) 
    <= model2.max_kgs)
model2.sales_constraint = pyo.Constraint(
    expr=sum(model2.sales[p] * model2.production[p] for p in model2.products) 
    <= model2.sales_limit)

# define objective
model2.profit_obj = pyo.Objective(
    expr=sum(model2.profit[p] * model2.production[p] for p in model2.products), 
    sense=pyo.maximize)

- A bit more flexible and compact (if we had them, defining constraints over indices would also be very easy)
- Easier to modify

In [9]:
# solve
opt = appsi.solvers.Highs()
results = opt.solve(model2)

# output some results
print(f"{model2.name}")
print(f"Status: {results.termination_condition}\n")
print(f"Total profit: {np.round(model2.profit_obj(), 2)}€")

for p in coefficients.keys():
    print(f"# {p}: {model2.production.get_values()[p]}")

bread&pizza_model2
Status: TerminationCondition.optimal

Total profit: 296.0€
# loafs: 7.0
# trays: 12.0


### Model 3 · Abstract model with rules

In [10]:
model3 = pyo.AbstractModel()

# "model first" approach, so Set and Param use empty declarations
model3.products = pyo.Set()

model3.max_hours = pyo.Param(within=pyo.NonNegativeReals)
model3.max_kgs = pyo.Param(within=pyo.NonNegativeReals)
model3.sales_limit = pyo.Param(within=pyo.NonNegativeReals)
model3.products_init = pyo.Param(within=pyo.NonNegativeReals)
model3.products_lb = pyo.Param(within=pyo.NonNegativeReals)
model3.products_ub = pyo.Param(within=pyo.NonNegativeReals)

model3.hours = pyo.Param(model3.products, within=pyo.NonNegativeReals) 
model3.flour = pyo.Param(model3.products, within=pyo.NonNegativeReals)
model3.sales = pyo.Param(model3.products, within=pyo.Reals)
model3.profit = pyo.Param(model3.products, within=pyo.Reals)

In [11]:
# define variables
model3.production = pyo.Var(
    model3.products, domain=pyo.NonNegativeIntegers, 
    initialize=model2.products_init, 
    bounds=(model2.products_lb, model2.products_ub)
)

**Rules**: declare external functions to define expressions

In [12]:
# define rules for the constraints
def time_constraint_rule(m):
    # instead of looping over p in m.products
    return pyo.summation(m.hours, m.production) <= m.max_hours

def flour_constraint_rule(m):  # could take several dimensions
    return pyo.summation(m.flour, m.production) <= m.max_kgs

def sales_constraint_rule(m):
    return pyo.summation(m.sales, m.production) <= m.sales_limit

model3.time_constraint = pyo.Constraint(rule=time_constraint_rule)
model3.flour_constraint = pyo.Constraint(rule=flour_constraint_rule)
model3.sales_constraint = pyo.Constraint(rule=sales_constraint_rule)

In [13]:
# rule for the objective
def profit_obj_rule(m):
    return pyo.summation(m.profit, m.production)

model3.profit_obj = pyo.Objective(rule=profit_obj_rule, sense=pyo.maximize)

- Rules can **include complex logics** on what is to be returned, allowing more control
- They are the standard way to approach constraints and objective definition 

**Data and instantiation**, just before solving the model

(see [here](https://pyomo.readthedocs.io/en/stable/working_abstractmodels/data/raw_dicts.html) for structure)

In [14]:
# define data separately
abstract_data = {None: {
    "products": {None: ['loafs', 'trays']},
    "max_hours": {None: 120},
    "max_kgs": {None: 50},
    "sales_limit": {None: 0},
    "hours": {'loafs': 8.5, 'trays': 4},
    "flour": {'loafs': 1.8, 'trays': 3},
    "sales": {'loafs': -2.00, 'trays': 1.00},
    "profit": {'loafs': 8, 'trays': 20},
    "products_init": {None: 0},
    "products_lb": {None: 0},
    "products_ub": {None: 100}
}}

# load data into an instance
instance = model3.create_instance(abstract_data)

In [15]:
# solve
opt = appsi.solvers.Highs()
results = opt.solve(instance)  
# refer to the instance for solve and results extraction

# output some results
print(f"Status: {results.termination_condition}\n")
print(f"Total profit: {np.round(instance.profit_obj(), 2)}€")

for p in instance.products:
    print(f"# {p}: {instance.production.extract_values()[p]}")

Status: TerminationCondition.optimal

Total profit: 296.0€
# loafs: 7.0
# trays: 12.0


- An abstract model is not so different from a concrete model, after all: what you choose is a preference
- **Rules** for constraints and objective are what allow complexity, flexibility and reusability

## 6. What else can I do?

- [Complex expressions](https://pyomo.readthedocs.io/en/stable/developer_reference/expressions/performance.html) and implementation of
    - linear, quadratic and general nonlinear terms
    - utility functions: `prod`, `quicksum`, `sum_product`
- Handling of **different classes of optimization models**, including integer and nonlinear ones

In terms of optimization process:
- Further inspection of results (slack, duals, values, ...) and model (can be saved to file)
- Iterations in solution phase: add/remove constraints, change parameters, activate/disactivate objective components, ...
- Set manipulations (especially useful for network problems)
- Some solver parameters modification (depends on each solver interface)

... and so much more! 

## 7. Considerations from my short experience

**Pros**
- Different options to reflect different levels of development \
  _&#8594; how I progress when prototyping a model_
- Tools that allow scalability and reusability \
  _&#8594; `AbstractModel` for clarity and reusability of components across different models_
- Several [modeling extensions](https://pyomo.readthedocs.io/en/stable/modeling_extensions/index.html) available via other packages (e.g., for bilevel, disjunctive, stochastic optimization + graphs and blocks for network problems)
- Multiple solvers options
- Modular design which encourages extensions \
  _&#8594; active community since 2008_
- Integration with all the rest of your Python workflow (e.g., for data processing, visualization, predictions models, ...)

**Cons**
- Documentation is not extensive \
  _&#8594; I often relied on StackOverflow and community posts to discover how to do things_
- Slight overhead?
- Debugging is not so easy (+ solver support is missing)
- Not very clear best practices / standards

## Resources

- [Pyomo documentiation](https://pyomo.readthedocs.io/en/stable/index.html#pyomo-documentation-release)
- Slides from the [Pyomo workshop 2023](https://github.com/Pyomo/pyomo-tutorials/blob/main/Pyomo-Workshop-December-2023.pdf)
- I've adapted the production mix model examples from this [tutorial](https://www.solvermax.com/blog/python-optimization-rosetta-stone), which is much more extensive and compares different Pyomo formulations with other ones (PuLP, OR Tools, SciPy, ...)

### _... I hope you'll try it!_

### Questions?